In [30]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

# Read in data

In [31]:
df = pd.read_csv('data/Australian Vehicle Prices.csv')

df.head()

,Brand,Year,Model,Car/Suv,Title,UsedOrNew,Transmission,Engine,DriveType,FuelType,FuelConsumption,Kilometres,ColourExtInt,Location,CylindersinEngine,BodyType,Doors,Seats,Price
0,Ssangyong,2022.0,Rexton,Sutherland Isuzu Ute,2022 Ssangyong Rexton Ultimate (awd),DEMO,Automatic,"4 cyl, 2.2 L",AWD,Diesel,8.7 L / 100 km,5595,White / Black,"Caringbah, NSW",4 cyl,SUV,4 Doors,7 Seats,51990
1,MG,2022.0,MG3,Hatchback,2022 MG MG3 Auto Excite (with Navigation),USED,Automatic,"4 cyl, 1.5 L",Front,Premium,6.7 L / 100 km,16,Black / Black,"Brookvale, NSW",4 cyl,Hatchback,5 Doors,5 Seats,19990
2,BMW,2022.0,430I,Coupe,2022 BMW 430I M Sport,USED,Automatic,"4 cyl, 2 L",Rear,Premium,6.6 L / 100 km,8472,Grey / White,"Sylvania, NSW",4 cyl,Coupe,2 Doors,4 Seats,108988
3,Mercedes-Benz,2011.0,E500,Coupe,2011 Mercedes-Benz E500 Elegance,USED,Automatic,"8 cyl, 5.5 L",Rear,Premium,11 L / 100 km,136517,White / Brown,"Mount Druitt, NSW",8 cyl,Coupe,2 Doors,4 Seats,32990
4,Renault,2022.0,Arkana,SUV,2022 Renault Arkana Intens,USED,Automatic,"4 cyl, 1.3 L",Front,Unleaded,6 L / 100 km,1035,Grey / Black,"Castle Hill, NSW",4 cyl,SUV,4 Doors,5 Seats,34990


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16734 entries, 0 to 16733
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Brand              16733 non-null  object 
 1   Year               16733 non-null  float64
 2   Model              16733 non-null  object 
 3   Car/Suv            16706 non-null  object 
 4   Title              16733 non-null  object 
 5   UsedOrNew          16733 non-null  object 
 6   Transmission       16733 non-null  object 
 7   Engine             16733 non-null  object 
 8   DriveType          16733 non-null  object 
 9   FuelType           16733 non-null  object 
 10  FuelConsumption    16733 non-null  object 
 11  Kilometres         16733 non-null  object 
 12  ColourExtInt       16733 non-null  object 
 13  Location           16284 non-null  object 
 14  CylindersinEngine  16733 non-null  object 
 15  BodyType           16452 non-null  object 
 16  Doors              151

# Preprocess the data

In [33]:
# Separate the `Engine column` to 'Engine_cylinder_number' and 'Engine_total_volume' 
df[['Engine_cylinder_number', 'Engine_total_volume']] = df['Engine'].str.split(',', expand=True)

In [34]:
# Clean values for `Engine_cylinder_number` and `Engine_total_volume` columns
df['Engine_cylinder_number'] = df['Engine_cylinder_number'].str.replace('cyl', '').str.replace('0 L', '').str.replace('2 L', '').str.replace('-', '')
df['Engine_total_volume'] = df['Engine_total_volume'].str.replace('L', '')
df = df.fillna(np.nan)

In [35]:
# Change ColourExtInt to only ExteriorColour
df['ExteriorColour'] = df['ColourExtInt'].str.split('/').str[0]
df['ExteriorColour'].value_counts()

ExteriorColour
White         5988
Silver        2316
Grey          2262
Black         1937
Blue          1577
Red           1130
-              395
5 years        269
Green          154
Gold           146
Brown          143
Orange         109
3 years         96
Yellow          87
6 years         31
7 years         25
2 years         13
4 years         11
Beige           10
Burgundy         9
Purple           9
Bronze           8
1 years          4
Turquoise        1
Other            1
Pink             1
Cream            1
Name: count, dtype: int64

Somehow the column "ExteriorColour" contains some "years" and "-" data, which doesn't make sense, so we will set these data to "Other".

In [36]:
# Clean values for `ExteriorColour` column
df.loc[df['ExteriorColour'].str.contains("years", na=False), 'ExteriorColour'] = "Other"
df['ExteriorColour'] = df['ExteriorColour'].str.replace('-', 'Other')

In [37]:
# Separate the `FuelConsumption` to `fuel_comsumption_liter` and `fuel_comsumption_km` 
df[['fuel_comsumption_liter', 'fuel_comsumption_km']] = df['FuelConsumption'].str.split('/', expand=True)

In [38]:
# Clean values for `kilometers` column
df['Kilometres'] = df['Kilometres'].str.replace('-', '').str.replace('/', '').str.replace(' ', '')
df['Kilometres'].replace('', np.nan, inplace=True)
df['Kilometres'] = df['Kilometres'].astype(float)

In [39]:
# Clean values for `price` column
df['Price'].replace('POA','', inplace=True)
df = df.replace('', np.nan)
df['Price'] = df['Price'].astype(float)

In [40]:
# Clean `Transmission` column
df['Transmission'].replace('-','', inplace=True)
df = df.replace('', np.nan)

In [41]:
# Clean `Year` column
df['Year'] = df['Year'].astype(str).str[:4]
df = df.replace('nan', np.nan)
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

In [42]:
df['Doors'].value_counts()

Doors
 4 Doors     11191
 5 Doors      2533
 2 Doors      1122
 3 Doors       213
 5 Seats        22
 2 Seats        10
 6 Seats         9
 7 Seats         8
 8 Seats         8
 3 Seats         6
 9 Seats         4
 4 Seats         3
 12 Seats        1
Name: count, dtype: int64

We found that in the column "Doors", some data contains "Seats", which is misplaced here and should be move to the "Seats" column.

In [43]:
# Clean values for Doors column
df['Doors'] = df['Doors'].str.replace('Doors', '')

for index, row in df.iterrows():
    # Check if the "Doors" column contains "Seats"
    if 'Seats' in str(row['Doors']):
        # Move to "Seats" column
        df.at[index, 'Seats'] = row['Doors']
        # Clear the "Doors" column for this row
        df.at[index, 'Doors'] = np.nan  # Set to None or an appropriate value

# Display unique values in the "Doors" column after the operation
df['Doors'].value_counts()


Doors
 4     11191
 5      2533
 2      1122
 3       213
Name: count, dtype: int64

Now that we move those misplaced data, let's check on the "Seats" column.

In [44]:
# Clean values for `Seats` column 
df['Seats'] = df['Seats'].str.replace('Seats', '')


In [45]:
# Clean values for `Body` column
df['BodyType'] = df['BodyType'].str.replace('/', '')

In [46]:
df['fuel_comsumption_km'].unique()

array([' 100 km', None], dtype=object)

In [47]:
# Clean values for `fuel_comsumption_liter` and `fuel_comsumption_km` columns
df['fuel_comsumption_liter'] = df['fuel_comsumption_liter'].str.replace('L', '')
df['fuel_comsumption_km'] = df['fuel_comsumption_km'].str.replace('km', '')
df['fuel_comsumption_liter'] = df['fuel_comsumption_liter'].str.replace('-', '')
df = df.replace('', np.nan)
df['fuel_comsumption_liter'] = df['fuel_comsumption_liter'].astype(float)
df['fuel_comsumption_km'] = df['fuel_comsumption_km'].astype(float)

In [48]:
df['FuelType'].unique()

array(['Diesel', 'Premium', 'Unleaded', 'Hybrid', '-', 'Other',
       'Electric', 'LPG', 'Leaded', nan], dtype=object)

In [49]:
# Clean values for 'FuelType' column
df['FuelType'] = df['FuelType'].str.replace('-', 'Other')

# Drop columns

In [50]:
drop_columns = ['Car/Suv', # Not useful
                'Title', # Not useful
                'Model', # Not useful
                'Location', # Not useful
                'Engine', # Seprated to 'Engine_cylinder_number' and 'Engine_total_volume'
                'FuelConsumption',  # Changed to `fuel_comsumption_liter_per100km`
                'ColourExtInt', # Changed to `ExteriorColour`
                'Location', # Not useful
                'CylindersinEngine', # Duplicated with `Engine_cylinder_number`  
]

In [51]:
# Drop the columns from the DataFrame
df.drop(columns=drop_columns, inplace=True)

In [52]:
df.head()

,Brand,Year,UsedOrNew,Transmission,DriveType,FuelType,Kilometres,BodyType,Doors,Seats,Price,Engine_cylinder_number,Engine_total_volume,ExteriorColour,fuel_comsumption_liter,fuel_comsumption_km
0,Ssangyong,2022.0,DEMO,Automatic,AWD,Diesel,5595.0,SUV,4,7,51990.0,4,2.2,White,8.7,100.0
1,MG,2022.0,USED,Automatic,Front,Premium,16.0,Hatchback,5,5,19990.0,4,1.5,Black,6.7,100.0
2,BMW,2022.0,USED,Automatic,Rear,Premium,8472.0,Coupe,2,4,108988.0,4,2,Grey,6.6,100.0
3,Mercedes-Benz,2011.0,USED,Automatic,Rear,Premium,136517.0,Coupe,2,4,32990.0,8,5.5,White,11.0,100.0
4,Renault,2022.0,USED,Automatic,Front,Unleaded,1035.0,SUV,4,5,34990.0,4,1.3,Grey,6.0,100.0


# Deal with nan values

In [53]:
# Find the percentage of nan value in each column
df.isnull().sum()/df.shape[0]*100

Brand                      0.005976
Year                       0.005976
UsedOrNew                  0.005976
Transmission               1.505916
DriveType                  0.005976
FuelType                   0.005976
Kilometres                 3.537708
BodyType                   1.685192
Doors                     10.009561
Seats                      9.764551
Price                      0.316720
Engine_cylinder_number    10.648978
Engine_total_volume       10.654954
ExteriorColour             0.005976
fuel_comsumption_liter    10.147006
fuel_comsumption_km       10.147006
dtype: float64

In [54]:
# Extract numerical columns from the DataFrame
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
# Create a DataFrame with only numerical columns
numerical_df = df[numerical_columns]
# Initialize a KNN imputer with 5 neighbors
knnimputer = KNNImputer(n_neighbors=5)  
# Apply the KNN imputer to the numerical DataFrame
numerical_cln_df = knnimputer.fit_transform(numerical_df)  
# Convert the imputed array back to a pandas DataFrame
numerical_cln_df = pd.DataFrame(numerical_cln_df, columns=numerical_df.columns)

# Extract categorical columns from the DataFrame
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
# Create a DataFrame with only categorical columns
categorical_df = df[categorical_columns]
# Initialize a simple imputer for categorical data using the most frequent strategy
categorical_imputer = SimpleImputer(strategy='most_frequent')
# Apply the simple imputer to the categorical DataFrame
categorical_cln_df = pd.DataFrame(categorical_imputer.fit_transform(categorical_df), columns=categorical_df.columns)

df=pd.merge(categorical_cln_df,numerical_cln_df,right_index=True,left_index=True)

In [55]:
df.isnull().sum()/df.shape[0]*100

Brand                     0.0
UsedOrNew                 0.0
Transmission              0.0
DriveType                 0.0
FuelType                  0.0
BodyType                  0.0
Doors                     0.0
Seats                     0.0
Engine_cylinder_number    0.0
Engine_total_volume       0.0
ExteriorColour            0.0
Year                      0.0
Kilometres                0.0
Price                     0.0
fuel_comsumption_liter    0.0
fuel_comsumption_km       0.0
dtype: float64

In [56]:
df['Year'] = df['Year'].astype(int)

In [57]:
df.sample(10)

,Brand,UsedOrNew,Transmission,DriveType,FuelType,BodyType,Doors,Seats,Engine_cylinder_number,Engine_total_volume,ExteriorColour,Year,Kilometres,Price,fuel_comsumption_liter,fuel_comsumption_km
2463,Holden,USED,Automatic,Rear,Unleaded,Sedan,4,5,6,3.6,White,2015,190878.0,19995.0,9.5,100.0
12792,Nissan,USED,Automatic,Front,Unleaded,SUV,4,5,4,2.5,White,2014,176895.0,17990.0,7.9,100.0
1577,Hyundai,USED,Automatic,Rear,Diesel,Wagon,4,8,4,2.5,White,2014,150302.0,21995.0,9.0,100.0
8315,Toyota,USED,Automatic,Front,Unleaded,Sedan,4,5,4,2.5,Brown,2014,213913.0,15999.0,7.8,100.0
3827,Porsche,USED,Automatic,AWD,Premium,SUV,4,5,6,2.9,White,2020,14249.0,147999.0,10.0,100.0
86,Jeep,USED,Automatic,Front,Premium,SUV,4,5,4,1.4,Grey,2017,69218.0,24950.0,5.9,100.0
11767,Hyundai,USED,Automatic,Front,Unleaded,Sedan,4,5,4,1.8,White,2011,136400.0,12990.0,7.1,100.0
16298,Nissan,USED,Automatic,Front,Unleaded,SUV,4,5,4,2,White,2012,86454.0,19990.0,8.2,100.0
3332,Kia,USED,Automatic,Front,Unleaded,SUV,4,5,4,2,Silver,2014,113640.0,20999.0,8.8,100.0
3012,Mitsubishi,USED,Automatic,4WD,Diesel,Ute Tray,4,5,4,2.4,White,2020,71670.0,42998.0,8.6,100.0


In [58]:
df.to_csv('data/preprocessed_data.csv', index=False)